In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from math import log2
%matplotlib inline

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.pipeline import make_pipeline
import time
import matplotlib as mpl
from sklearn.ensemble import RandomForestClassifier as RFC


from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,QuantileTransformer,normalize
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

import pathlib
import os
import sys
import functions

In [2]:
# Custom functions
import pathlib
import os
import sys

module_path = pathlib.Path(os.getcwd()).parent
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
print(sys.path)

from common.dataset_grabber import get_data_path

['/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex1/amazon', '', '/home/alexander/ngsuite/ngsolve-install/lib/python3/dist-packages', '/home/alexander/ngsuite/ngsolve-install/bin', '/home/alexander/.local/bin', '/usr/local/sbin', '/usr/local/bin', '/usr/sbin', '/usr/bin', '/sbin', '/bin', '/usr/games', '/usr/local/games', '/snap/bin', '/home/alexander/.vscode/extensions/ms-python.python-2020.10.332292344/pythonFiles', '/home/alexander/.vscode/extensions/ms-python.python-2020.10.332292344/pythonFiles/lib/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/home/alexander/.local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/home/alexander/.local/lib/python3.7/site-packages/IPython/extensions', '/home/alexander/.ipython', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex1']


In [3]:
datapath_train = get_data_path("Amazon_Review_Data", "amazon_review_ID.shuf.lrn.csv")
display(datapath_train)
datapath_test = get_data_path("Amazon_Review_Data", "amazon_review_ID.shuf.tes.csv")
display(datapath_test)

PosixPath('/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex1/Datasets/Amazon_Review_Data/amazon_review_ID.shuf.lrn.csv')

PosixPath('/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex1/Datasets/Amazon_Review_Data/amazon_review_ID.shuf.tes.csv')

In [4]:
df_train = pd.read_csv(datapath_train).drop("ID",axis = 1)
df_test = pd.read_csv(datapath_test).drop("ID",axis = 1)

df_train

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V9992,V9993,V9994,V9995,V9996,V9997,V9998,V9999,V10000,Class
0,8,6,10,6,7,2,2,2,3,2,...,1,0,0,0,0,0,0,0,0,Chell
1,13,1,13,9,8,8,2,3,2,4,...,4,1,2,1,0,1,0,0,0,Engineer
2,16,7,6,7,9,3,4,2,6,5,...,0,0,0,0,1,0,0,0,0,Grove
3,8,11,10,11,3,7,0,4,2,1,...,0,0,1,0,0,0,2,1,0,Davisson
4,10,11,8,5,3,4,2,5,5,4,...,0,0,0,0,0,0,0,0,0,Wilson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,9,8,5,5,6,2,14,1,4,5,...,0,1,0,0,1,0,1,0,0,Riley
746,6,13,4,6,8,2,7,3,3,2,...,0,0,0,0,1,0,0,0,1,Neal
747,16,11,4,9,9,4,0,3,5,0,...,0,0,0,0,0,1,1,0,4,Messick
748,9,5,4,2,3,7,2,7,1,1,...,0,0,0,0,0,2,0,0,0,Mitchell


In [5]:
X_lrn = df_train.loc[:,'V1':'V10000']
Y_lrn = df_train.loc[:,'Class':'Class']

In [ ]:
def plot_corr_heatmap(df, fmt=".2f", feat_to_ret="Class", ticksfont=12,abs = True):
    Class_description = []
    Class_description = pd.unique(df.Class)

    for i in range(len(Class_description)):
        df.Class[df.Class == Class_description[i]] = i+1
    
    plt.rcParams.update({'font.size': 14, 'font.weight': 'bold'})
    # Compute correlations and save in matrix
    if abs:
        corr = np.abs(df.corr()) # We only used absolute values for visualization purposes! ..."hot-cold" view to just sort between 
    else:
        corr = df.corr()

    # Mask the repeated values --> here: upper triangle

    #print(corr)
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True # mask upper triangle

    corr_to_feat = corr.loc[:,feat_to_ret]
    
    f, ax = plt.subplots(figsize=(18, 16))
    sns.heatmap(corr, annot=True, fmt=fmt , mask=mask, vmin=0, vmax=1, linewidths=.5,cmap="YlGnBu")
    plt.tick_params(labelsize=ticksfont)
    return corr_to_feat

# Feature selection

## varianz selection

In [ ]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_lrn_var = sel.fit_transform(X_lrn)
pd.DataFrame(X_lrn_var)

In [ ]:
k = 4

## k-best selection

In [ ]:


#num_features = X_lrn.shape[1]
#k= int(sqrt(num_features)*1.7)
X_lrn_kb = SelectKBest(chi2, k=k).fit_transform(X_lrn, Y_lrn)
X_lrn_kb.shape

In [ ]:
sns.pairplot(pd.DataFrame(X_lrn_kb))

## PCA

In [ ]:
pca = PCA(n_components=k)
X_lrn_PCA = pca.fit(X_lrn.values).transform(X_lrn.values)
X_lrn_PCA.shape
pd.DataFrame(X_lrn_PCA)

In [ ]:
sns.pairplot(pd.DataFrame(X_lrn_PCA))

In [ ]:
corr_to_class_stripped = functions.plot_corr_heatmap(X_lrn_PCA, ticksfont=16,abs = False)

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_lrn, Y_lrn, test_size=.4)
X_train.shape

In [ ]:
scaler = StandardScaler()  
scaler.fit(X_train) 
X_train_SC = scaler.transform(X_train) 
X_valid_SC = scaler.transform(X_valid) 

In [ ]:
g = 100
start = time.time()
#X_train, df_test = functions.DefinScaler(best_params_SC["scaler"], X_train, df_test)
clf = MLPClassifier(hidden_layer_sizes=(g,g),alpha=0.1,activation="relu",solver="adam",max_iter=200)
clf.fit(X_train_SC, Y_train)
Y_pred = clf.predict(X_valid_SC)
ende = time.time()
print(accuracy_score(Y_valid, Y_pred),'{:5.3f}s'.format(ende-start))


In [ ]:
def MLP_selection(h, X_lrn, Y_lrn):
    X_train, X_valid, Y_train, Y_valid = train_test_split(X_lrn, Y_lrn, test_size=.4)
    scaler = StandardScaler()  
    scaler.fit(X_train) 
    X_train_SC = scaler.transform(X_train) 
    X_valid_SC = scaler.transform(X_valid)  
    clf = MLPClassifier(hidden_layer_sizes=(h,h),alpha=0.1,activation="relu",solver="adam",max_iter=200)
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_valid)
    score = accuracy_score(Y_valid, Y_pred)
    print("MLP: ", "score: ",score)
    return score

def RF_selection(X_lrn, Y_lrn):
    rf_shallow = RFC(n_jobs=4, n_estimators=2000, max_features=0.02,bootstrap=True, oob_score=True, max_depth=60, min_samples_leaf=4, min_samples_split=4,verbose=0)
    X_train, X_valid, Y_train, Y_valid = train_test_split(X_lrn, Y_lrn, test_size=.4)
    scaler = StandardScaler()  
    scaler.fit(X_train) 
    X_train_SC = scaler.transform(X_train) 
    X_valid_SC = scaler.transform(X_valid)    

    rf_shallow.fit(X_train, Y_train)
    Y_pred = rf_shallow.predict(X_valid)
    score = accuracy_score(Y_valid, Y_pred)
    print("RF: ", "score: ",score)
    return score


    #print(accuracy_score(Y_valid, Y_pred),'{:5.3f}s'.format(ende-start))
def FeatureSelection(selector, k, X_lrn, Y_lrn):
    if selector == "KBest":
        df = pd.DataFrame(SelectKBest(chi2, k=k).fit_transform(X_lrn, Y_lrn))
    if selector == "PCA":
        pca = PCA(n_components=k)
        df = pca.fit(X_lrn.values).transform(X_lrn.values)

    return df

def CompairFeatueSelection(mod,selector, k, X_lrn, Y_lrn,h):
    X_lrn_FS = FeatureSelection(selector, k, X_lrn, Y_lrn)
    if mod == "MLP":
        score = MLP_selection(h, X_lrn_FS, Y_lrn)
    if mod == "RF":
        score = RF_selection(X_lrn_FS, Y_lrn)
    #if mod == "KNN":
    #    score = MLP_selection(h, X_lrn_FS, Y_lrn)
    #print("k: ",k,"h:",h , "score: ",score)
    return score



In [ ]:
score_list = []
k = [10,20,30,50,100,200,300,400,500,1000,1500]
k = [200,300,400,500,700,1000,1100,1200,1300,1500]
hidden = 200
selector = ["KBest","PCA"]
model = ["MLP", "RF"]
for select in selector:
    for mod in model:
        score_list = []
        for ki in k:
            score = CompairFeatueSelection(mod, select, ki, X_lrn, Y_lrn, hidden)
            score_list.append(score)
        plt.figure()
        plt.grid()
        plt.title("{} {}".format(select,mod))
        plt.plot(k,score_list)

# Training

## RandomForrest 

In [9]:
rf_shallow = RFC(n_jobs=4, n_estimators=10000, max_features=0.02,bootstrap=True, oob_score=True, max_depth=60, min_samples_leaf=4, min_samples_split=4,verbose=1000)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_lrn, Y_lrn, test_size=.4)
scaler = StandardScaler()  
scaler.fit(X_train) 
X_train_SC = scaler.transform(X_train) 
X_valid_SC = scaler.transform(X_valid)

rf_shallow.fit(X_train, Y_train)
Y_pred = rf_shallow.predict(X_valid)
score = accuracy_score(Y_valid, Y_pred)
print("RF: ", "score: ",score)

l(n_jobs=4)]: Done 9673 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9674 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9675 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9676 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9677 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9678 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9679 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9680 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9681 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9682 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9683 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9684 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9685 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9686 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9687 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 9688 tasks      | elapsed:    6.7s
[Parallel(n_jobs

# Validation

In [10]:
df_test = pd.read_csv(datapath_test).drop("ID",axis = 1)

In [ ]:
#VT = VarianceThreshold(threshold=0.05)
features = df_test.loc[:,'V1':'V10000']
#target = df_test.loc[:,'Class']
num_features = features.shape[1]
features

# Create and fit selector
k= int(sqrt(num_features)*1.7)
print(k)
selector = SelectKBest(k=k)
# Get columns to keep and create new dataframe with those only
cols = selector.fit(features.values, target.values).get_support(indices=True)
df_test = features.iloc[:,cols]

In [11]:
scaler = StandardScaler()  
scaler.fit(df_test) 
df_test_SC = scaler.transform(df_test) 

In [12]:
Y_pred = clf.predict(df_test_SC)

NameError: name 'clf' is not defined

In [13]:
Y_pred = rf_shallow.predict(df_test_SC)

     | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9673 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9674 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9675 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9676 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9677 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9678 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9679 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9680 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9681 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9682 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9683 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9684 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9685 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9686 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9687 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 9688 tasks      | e

# Filepreparation for the Kagglecompetition

In [14]:
solution_data = list(zip(list(range(750,1500)), Y_pred))

In [22]:
solution_table = pd.DataFrame(solution_data, columns=['ID', 'Class'])
solution_table.Class

0      Blankenship
1          Chachra
2          Chachra
3          Chachra
4          Chachra
          ...     
745        Chachra
746        Chachra
747        Chachra
748        Chachra
749        Chachra
Name: Class, Length: 750, dtype: object

In [16]:
solution_table.to_csv(f"results/SC_MLP.csv", index=False)